### Use this notebook for developing mlframe prototype 2

In [8]:
import multiprocessing
from Queue import Queue, Empty
import subprocess
import os
import time
import re
import sys

In [102]:
# Class for executing commands in background processes.
# command_and_args can be a list or a string with the command and arguments separated with spaces.
# It can be used for executing remote processes in background. For that purpose exec_remote.sh should be used.
# Usage sample: 
# d = BashExecutor(command,hostname=hostname)
# d.start()
# exec_remote.sh must output subprocess exit code in the form:
# exitcode=N
# , where N is the number.
class BashExecutor(multiprocessing.Process):
    
    def __init__(self, command_and_args, debug=False, hostname=""):
        super(BashExecutor,self).__init__()
        self.debug = debug
        self.hostname = hostname
        if self.debug: print "command args:",command_and_args
        if type(command_and_args) is str:
            self.command = command_and_args.split(" ")
        else:
            self.command=command_and_args
        self.exitcode_pat = re.compile("^exitcode=(\d+)")
        manager = multiprocessing.Manager()
        d = manager.dict()
        self.ec = None
        d[1]="one"
        self.d = d
        
        
    # Poll exit code of self.proc and store it if not None.
    def poll(self):
        exitcode = self.proc.poll()
#         if self.debug: 
#             if exitcode is not None:
#                 print "polled exitcode=",exitcode
        if exitcode is not None:
            self.setExitCode(exitcode)
        return exitcode
    
    
    # Set given exit code to Command class object
    def setExitCode(self, ec):
        if "exitcode" not in self.d:
            if self.debug: print "setting exit code to",ec        
            self.d["exitcode"] = ec
            if False: #isinstance(self.command, Command): 
                if self.command.exitcode is None:
                    self.command.exitcode = ec
                if self.debug:
                    print "Set exit code to",ec
                    if ec is not None:
                        printObj(self.command)
    
    
    # Copy process stdout to self.stdout queue line by line.
    def __handleOutput__(self, q):
        proc = self.proc   
        q.put("First line to stdout")
        for line in iter(proc.stdout.readline, b''):
            if line is not None:
                m = self.exitcode_pat.match(line)
                if m is not None:
                    print "Exitcode match",m.group(0)
                q.put(line)
                if self.debug: 
                    print ">",self.name,line,
                sys.stdout.flush()
                time.sleep(1)
                
            #if line is not None and len(line)>0:
            #        print self.hostname,line,
                    
    # Copy process stderr to self.stderr queue line by line.
    def __handleError__(self, q):
        proc = self.proc
        for line in iter(proc.stderr.readline, b''):
            q.put(line)
            #if line is not None and len(line)>0:
            #        print "!",self.hostname,line,
            sys.stdout.flush()
        
    # Must be calles to start the process in background
    def run(self):
        if self.debug: 
            print "In {}. Calling {}".format(self.name,self.command)
            
        proc = subprocess.Popen(self.command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=1, shell=False)
        self.proc = proc
        out_q = multiprocessing.Queue()
        p_out = multiprocessing.Process(target=self.__handleOutput__,args=(out_q,))
        p_out.start()
        err_q = multiprocessing.Queue()
        p_err = multiprocessing.Process(target=self.__handleError__,args=(err_q,))
        p_err.start()
        if self.debug: print self.name, "process started"
        while self.poll() is None or not out_q.empty() or not err_q.empty():
            #if self.debug: 
            #    print self.poll(),out_q.empty(),err_q.empty()
            
            try:
                outline = out_q.get_nowait()
                errline = err_q.get_nowait()
                if self.debug:
                    if outline is not None:
                        print "OUT",self.name, outline,
                    if errline is not None:
                        print "ERR",self.name, errline,
            except Empty:
                if self.debug: print "-",
                time.sleep(1)
            
        try:
            line = out_q.get_nowait()                
            if self.debug:
                print "STDOUT",self.name,line
        except Empty:
            pass
        
        if self.debug:
            print "Exiting",multiprocessing.current_process().name,self.d["exitcode"]
        return

    def getExitcode(self):
        if self.debug: print "Should be one:",self.d[1]
        return self.d["exitcode"]

In [103]:
d1 = BashExecutor("/Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh",debug=True)
d1.start()
#d2 = RemoteExec("mouse","test_nvidia_ubuntu.sh", debug=True)
#RemoteExec("DL","test_nvidia_ubuntu.sh", debug=False)

command args: /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh
In BashExecutor-99. Calling ['/Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh']
BashExecutor-99 process started
> BashExecutor-99 Running test command on peters-imac.local
- OUT BashExecutor-99 First line to stdout ERR BashExecutor-99 peters-imac.local err
OUT BashExecutor-99 Running test command on peters-imac.local
ERR BashExecutor-99 peters-imac.local err
> BashExecutor-99 peters-imac.local():1
- setting exit code to 125
OUT BashExecutor-99 peters-imac.local():1
> BashExecutor-99 peters-imac.local():2
ERR BashExecutor-99 peters-imac.local err
OUT BashExecutor-99 peters-imac.local():2
ERR BashExecutor-99 peters-imac.local err
> BashExecutor-99 peters-imac.local():3
- OUT BashExecutor-99 peters-imac.local():3
ERR BashExecutor-99 peters-imac.local err
> BashExecutor-99 peters-imac.local():4
- OUT BashExecutor-99 peters-imac.local():4
ERR BashExecutor-99 peters-imac.local err
> BashExecutor

In [104]:
print d1.getExitcode()

Should be one: one
125
